In [ ]:
%pip install pycaret
%pip install optuna
%pip install optuna-integration[sklearn]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 4.3 MB/s eta 0:00:00


In [ ]:
from google.colab import files #匯入套件
import pandas as pd
upload = files.upload() #開啟上傳介面並載入

Saving test.csv to test.csv
Saving train.csv to train.csv


**Prompt:**

According to the code in this article, write a Python program that can compare Machine Learning agorithms on classification problem:
1. Not needed feature engineering for now.
2. Use the dataset "Titanic" on Kaggle, multi-features is accept.
3. Use library "Pycarat" for Machine Learning.
4. Use function 'compare_models(n_select=16)' that can select top 16 models to compare Machine Learning algorithms.
5. Use top 16 models for Stacking on classification problem.

In [ ]:
import pandas as pd
from pycaret.classification import setup, compare_models, stack_models, predict_model

# Load Titanic dataset
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# Drop rows with missing target values
train_data = data_train.dropna(subset=['Survived'])

# Define the setup for PyCaret
env = setup(
    data=train_data,
    target='Survived',
    categorical_features=['Pclass', 'Sex', 'Embarked'],
    numeric_features=['Age', 'Fare'],
    ignore_features=['PassengerId', 'Name', 'Ticket', 'Cabin']
)

# Compare and select top 16 models
best_models = compare_models(
    n_select=16,
    include=[
        'lr',      # Logistic Regression
        'knn',     # K Neighbors Classifier
        'nb',      # Naive Bayes
        'dt',      # Decision Tree Classifier
        'svm',     # SVM - Linear Kernel
        'rbfsvm',  # SVM - Radial Kernel
        'gpc',     # Gaussian Process Classifier
        'mlp',     # MLP Classifier
        'ridge',   # Ridge Classifier
        'rf',      # Random Forest Classifier
        'qda',     # Quadratic Discriminant Analysis
        'ada',     # Ada Boost Classifier
        'gbc',     # Gradient Boosting Classifier
        'lda',     # Linear Discriminant Analysis
        'et',      # Extra Trees Classifier
        'lightgbm',# Light Gradient Boosting Machine
    ]
)

# Perform Stacking using the top 16 models
stacked_model = stack_models(best_models)

# Predict on the test dataset
# Note: Test dataset needs similar preprocessing as training dataset. Adjust as necessary.
predictions = predict_model(stacked_model, data=data_test)

# Save the predictions
predictions[['PassengerId', 'prediction_label']].to_csv('titanic_predictions.csv', index=False)

print("Stacking complete. Predictions saved to 'titanic_predictions.csv'.")


,Description,Value
0,Session id,3365
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 12)"
4,Transformed data shape,"(891, 12)"
5,Transformed train set shape,"(623, 12)"
6,Transformed test set shape,"(268, 12)"
7,Ignore features,4
8,Numeric features,2
9,Categorical features,3


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
mlp,MLP Classifier,0.8088,0.8538,0.7069,0.7830,0.7369,0.5880,0.5951,0.8110
lightgbm,Light Gradient Boosting Machine,0.8088,0.8604,0.7152,0.7729,0.7389,0.5891,0.5935,0.1300
gbc,Gradient Boosting Classifier,0.8072,0.8610,0.6775,0.7900,0.7260,0.5797,0.5859,0.2790
rf,Random Forest Classifier,0.7960,0.8660,0.7150,0.7463,0.7230,0.5632,0.5690,0.2320
ridge,Ridge Classifier,0.7912,0.8481,0.6775,0.7544,0.7115,0.5491,0.5530,0.0750
et,Extra Trees Classifier,0.7897,0.8483,0.7111,0.7346,0.7152,0.5502,0.5560,0.2040
lda,Linear Discriminant Analysis,0.7896,0.8483,0.6775,0.7509,0.7099,0.5459,0.5497,0.1330
lr,Logistic Regression,0.7864,0.8475,0.6650,0.7525,0.7040,0.5382,0.5424,0.7430
dt,Decision Tree Classifier,0.7848,0.7699,0.7192,0.7223,0.7145,0.5431,0.5481,0.0740
ada,Ada Boost Classifier,0.7848,0.8209,0.7192,0.7233,0.7192,0.5449,0.5470,0.1530


Processing:   0%|          | 0/84 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8254,0.8985,0.7083,0.8095,0.7556,0.6207,0.6240
1,0.8730,0.9188,0.8333,0.8333,0.8333,0.7308,0.7308
2,0.8571,0.8552,0.7083,0.8947,0.7907,0.6845,0.6952
3,0.7581,0.8194,0.5652,0.7222,0.6341,0.4574,0.4651
4,0.7903,0.8454,0.7083,0.7391,0.7234,0.5547,0.5550
5,0.8387,0.9232,0.7500,0.8182,0.7826,0.6548,0.6564
6,0.8065,0.8942,0.7083,0.7727,0.7391,0.5857,0.5871
7,0.8548,0.9128,0.7917,0.8261,0.8085,0.6917,0.6921
8,0.7581,0.8103,0.7500,0.6667,0.7059,0.5016,0.5041


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

Stacking complete. Predictions saved to 'titanic_predictions.csv'.


**Prompt:**

According to code version1, do model optimization, meta-heuristic.
1. Include feature engineering in the article.
2. model selection (ensemble).
3. Use optuna to do tuning,
   "search_library='optuna',search_algorithm='random'" in function 'tune_model'.

In [ ]:
import pandas as pd
from pycaret.classification import setup, compare_models, stack_models, predict_model, tune_model, create_model

# Load Titanic dataset
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# Feature engineering
def simplify_ages(df):
    df['Age'] = df['Age'].fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
    df['Age'] = pd.cut(df['Age'], bins, labels=group_names)
    return df

def simplify_fares(df):
    df['Fare'] = df['Fare'].fillna(-0.5)
    bins = (-1, 0, 8, 15, 31, 1000)
    group_names = ['Unknown', '1_quartile', '2_quartile', '3_quartile', '4_quartile']
    df['Fare'] = pd.cut(df['Fare'], bins, labels=group_names)
    return df

def simplify_cabins(df):
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Cabin'] = df['Cabin'].apply(lambda x: x[0])
    return df

def format_name(df):
    df['Lname'] = df['Name'].apply(lambda x: x.split(' ')[0])
    df['NamePrefix'] = df['Name'].apply(lambda x: x.split(' ')[1])
    return df

def drop_features(df):
    return df.drop(['Ticket', 'Name', 'Embarked'], axis=1)

def transform_features(df):
    df = simplify_ages(df)
    df = simplify_fares(df)
    df = simplify_cabins(df)
    df = format_name(df)
    df = drop_features(df)
    return df

train_data = transform_features(data_train)
data_test = transform_features(data_test)

# Drop rows with missing target values
train_data = train_data.dropna(subset=['Survived'])

# Define the setup for PyCaret
env = setup(
    data=train_data,
    target='Survived',
    categorical_features=['Pclass', 'Sex', 'Age', 'Fare', 'Cabin', 'Lname', 'NamePrefix'],
    numeric_features=[],
    ignore_features=['PassengerId'],
)

# Compare and select top 16 models
best_models = compare_models(
    n_select=16,
    include=[
        'lr',      # Logistic Regression
        'knn',     # K Neighbors Classifier
        'nb',      # Naive Bayes
        'dt',      # Decision Tree Classifier
        'svm',     # SVM - Linear Kernel
        'rbfsvm',  # SVM - Radial Kernel
        'gpc',     # Gaussian Process Classifier
        'mlp',     # MLP Classifier
        'ridge',   # Ridge Classifier
        'rf',      # Random Forest Classifier
        'qda',     # Quadratic Discriminant Analysis
        'ada',     # Ada Boost Classifier
        'gbc',     # Gradient Boosting Classifier
        'lda',     # Linear Discriminant Analysis
        'et',      # Extra Trees Classifier
        'lightgbm',# Light Gradient Boosting Machine
    ]
)

# Optimize top models using Optuna
tuned_models = [tune_model(model, search_library='optuna', search_algorithm='random') for model in best_models]

# Perform Stacking using the optimized models
stacked_model = stack_models(tuned_models)

# Predict on the test dataset
predictions = predict_model(stacked_model, data=data_test)

# Save the predictions
predictions[['PassengerId', 'prediction_label']].to_csv('titanic_predictions_optimized.csv', index=False)

print("Optimization and stacking complete. Predictions saved to 'titanic_predictions_optimized.csv'.")


,Description,Value
0,Session id,3908
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 11)"
4,Transformed data shape,"(891, 54)"
5,Transformed train set shape,"(623, 54)"
6,Transformed test set shape,"(268, 54)"
7,Ignore features,1
8,Numeric features,2
9,Categorical features,7


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8137,0.8573,0.6904,0.8074,0.7371,0.5952,0.6056,0.1410
gpc,Gaussian Process Classifier,0.8058,0.8407,0.6866,0.7948,0.7254,0.5781,0.5911,0.2790
knn,K Neighbors Classifier,0.8011,0.8286,0.6484,0.8018,0.7086,0.5617,0.5754,0.1390
rbfsvm,SVM - Radial Kernel,0.8009,0.8401,0.6861,0.7785,0.7239,0.5699,0.5772,0.2190
ridge,Ridge Classifier,0.7977,0.8333,0.6650,0.7831,0.7114,0.5588,0.5693,0.1300
mlp,MLP Classifier,0.7769,0.7942,0.6322,0.7569,0.6799,0.5128,0.5242,1.2980
rf,Random Forest Classifier,0.7657,0.8106,0.6234,0.7322,0.6673,0.4897,0.4980,0.2820
et,Extra Trees Classifier,0.7546,0.7827,0.6192,0.7101,0.6563,0.4680,0.4744,0.2570
gbc,Gradient Boosting Classifier,0.7480,0.7269,0.6152,0.6955,0.6474,0.4542,0.4598,0.3820
ada,Ada Boost Classifier,0.7401,0.7384,0.6480,0.6786,0.6543,0.4478,0.4559,0.2400


Processing:   0%|          | 0/84 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8889,0.8980,0.7917,0.9048,0.8444,0.7586,0.7627
1,0.7460,0.7527,0.6250,0.6818,0.6522,0.4528,0.4538
2,0.8889,0.8809,0.7917,0.9048,0.8444,0.7586,0.7627
3,0.7742,0.8701,0.7917,0.6786,0.7308,0.5383,0.5430
4,0.7258,0.7758,0.5417,0.6842,0.6047,0.3991,0.4055
5,0.8226,0.7944,0.6250,0.8824,0.7317,0.6049,0.6249
6,0.9032,0.9545,0.9167,0.8462,0.8800,0.7991,0.8009
7,0.8065,0.8799,0.5417,0.9286,0.6842,0.5582,0.6004
8,0.8548,0.8498,0.7083,0.8947,0.7907,0.6819,0.6928


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:optuna_integration.sklearn.sklearn:Searching the best hyperparameters using 623 samples...
INFO:optuna_integration.sklearn.sklearn:Finished hyperparameter search!


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8254,0.8104,0.7500,0.7826,0.7660,0.6268,0.6272
1,0.7143,0.6971,0.5417,0.6500,0.5909,0.3742,0.3778
2,0.8413,0.8446,0.7083,0.8500,0.7727,0.6523,0.6587
3,0.7903,0.8218,0.7500,0.7200,0.7347,0.5615,0.5618
4,0.7419,0.7248,0.5000,0.7500,0.6000,0.4206,0.4394
5,0.8226,0.7780,0.7083,0.8095,0.7556,0.6173,0.6207
6,0.8065,0.8416,0.7917,0.7308,0.7600,0.5983,0.5996
7,0.7258,0.7752,0.4583,0.7333,0.5641,0.3793,0.4016
8,0.7742,0.8109,0.6250,0.7500,0.6818,0.5090,0.5141


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

Optimization and stacking complete. Predictions saved to 'titanic_predictions_optimized.csv'.
